# Imports 

In [1]:
from parse import *
from analyse import *
import pandas as pd
# from sklearn.model_selection import train_test_split as tts
# from itertools import chain
# import string
# import numpy as np



/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:81: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


# Declarations

In [2]:
%matplotlib inline

# Build the graph (optional vs reload)

In [3]:
G = parseText('data/electionActLabeled.txt', verbose=True)

--------------------------------------------------------------------------------
Heading: Interpretation
Last action: setSectionName
currentHeadings['Alberta_Election_Act', 'None', 0, 'None', 0, 'Interpretation']
--------------------------------------------------------------------------------
Heading: 1(1)
Last action: setSubSection
currentHeadings['Alberta_Election_Act', 'None', 0, 'None', 0, 'Interpretation', '1', '1']
--------------------------------------------------------------------------------
Heading: (a)
Last action: setSubSubSection
currentHeadings['Alberta_Election_Act', 'None', 0, 'None', 0, 'Interpretation', '1', '1', 'a']
--------------------------------------------------------------------------------
Heading: (b)
Last action: setSubSubSection
currentHeadings['Alberta_Election_Act', 'None', 0, 'None', 0, 'Interpretation', '1', '1', 'b']
--------------------------------------------------------------------------------
Heading: (b.1)
Last action: setSubSubSection
currentHead

--------------------------------------------------------------------------------
Heading: Secrecy
Last action: setSectionName
currentHeadings['Alberta_Election_Act', 'Elections, By‑Elections and Plebiscites', '3', 'None', '0', 'Secrecy']
--------------------------------------------------------------------------------
Heading: 93(1)
Last action: setSubSection
currentHeadings['Alberta_Election_Act', 'Elections, By‑Elections and Plebiscites', '3', 'None', '0', 'Secrecy', '93', '1']
--------------------------------------------------------------------------------
Heading: (2)
Last action: setSubSection
currentHeadings['Alberta_Election_Act', 'Elections, By‑Elections and Plebiscites', '3', 'None', '0', 'Secrecy', '93', '2']
--------------------------------------------------------------------------------
Heading: Preservation of peace and order
Last action: setSectionName
currentHeadings['Alberta_Election_Act', 'Elections, By‑Elections and Plebiscites', '3', 'None', '0', 'Preservation of peac

currentHeadings['Alberta_Election_Act', 'Post‑Polling‑Day Procedures', '4', 'None', '0', 'Conduct of official count', '137', '2']
--------------------------------------------------------------------------------
Heading: (a)
Last action: setSubSubSection
currentHeadings['Alberta_Election_Act', 'Post‑Polling‑Day Procedures', '4', 'None', '0', 'Conduct of official count', '137', '2', 'a']
--------------------------------------------------------------------------------
Heading: (b)
Last action: setSubSubSection
currentHeadings['Alberta_Election_Act', 'Post‑Polling‑Day Procedures', '4', 'None', '0', 'Conduct of official count', '137', '2', 'b']
--------------------------------------------------------------------------------
Heading: (c)
Last action: setSubSubSection
currentHeadings['Alberta_Election_Act', 'Post‑Polling‑Day Procedures', '4', 'None', '0', 'Conduct of official count', '137', '2', 'c']
--------------------------------------------------------------------------------
Heading: (d)

node: Alberta_Election_Act--Elections,_By‑Elections_and_Plebiscites--3--None--0--Polling_place_for_advance_poll--98--5:{'text': 'After the opening of a polling place for an advance poll the deputy returning officer shall take all reasonable precautions to ensure that no other person except the poll clerk has access to the ballot box.', 'level': 8, 'part': '3', 'partName': 'Elections, By‑Elections and Plebiscites', 'division': '0', 'divisionName': 'None', 'sectionName': 'Polling place for advance poll', 'section': '98', 'subsection': '5', 'subsubsection': '0', 'subsubsubsection': '0', 'subsubsubsubsection': '0', 'name': 'Alberta_Election_Act--Elections,_By‑Elections_and_Plebiscites--3--None--0--Polling_place_for_advance_poll--98--5'}
node: Alberta_Election_Act--Elections,_By‑Elections_and_Plebiscites--3--None--0--Polling_place_for_advance_poll--98--6:{'text': 'Each day at the close of the polling place for an advance poll the ballot box', 'level': 8, 'part': '3', 'partName': 'Elections,

# Load the prior data

In [4]:
train = pd.read_pickle('dataset/train2.pickle')
eval_data = pd.read_pickle('dataset/test2.pickle')
interpretations = pd.read_pickle('dataset/interpretations2.pickle')
G = nx.read_gpickle('dataset/graph2.pickle')

In [5]:
set_eval_data(eval_data)

# Spacy Evaluation

In [6]:
part_results = []
whole_results = []

for text in eval_data.text:
    wholes = set()
    parts = set()
    
    doc = nlp(text)
    for ent in doc.ents:
        wholes.add(ent.text)
        parts.update(ent.text.split())
    part_results.append(parts)
    whole_results.append(wholes)

r = evaluate_entity_sets(whole_results)
print()
rtok = evaluate_entity_token_sets(part_results)


Entity Set Precision: 0.2824427480916031
Entity Set Recall: 0.06642728904847396
Entity Set F1: 0.1075581395348837

Entity Token Set Precision: 0.7834645669291339
Entity Token Set Recall: 0.19490695396669933
Entity Token Set F1: 0.3121568627450981


# Processing

In [7]:
## the seeds in this case are a collection of pattern object built from the initial interpretation section in the graph
seeds = Patterns({x for x in interpretations.seed if x})

The Patterns class has an evaluation function built into it which runs the seeds in their current state against the loaded evaluation data, and returns an evaluation object which contains the details of the eval.

In its current state, the patterns are just from the parsed interpretation section

In [8]:
seeds.evaluate()

Entity Set Precision: 0.7154471544715447
Entity Set Recall: 0.31597845601436264
Entity Set F1: 0.4383561643835616
Entity Token Set Precision: 0.9510309278350515
Entity Token Set Recall: 0.3614103819784525
Entity Token Set F1: 0.5237757274662882


({'precision': 0.7154471544715447,
  'recall': 0.31597845601436264,
  'f1': 0.4383561643835616,
  'results': [{'correct': 1, 'false_positive': 0, 'false_negative': 3},
   {'correct': 0, 'false_positive': 1, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 1, 'false_positive': 0, 'false_negative': 0},
   {'correct': 1, 'false_positive': 0, 'false_negative': 5},
   {'correct': 1, 'false_positive': 0, 'false_negative': 0},
   {'correct': 0, 'false_positive': 0, 'false_negative': 3},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 0},
   {'correct': 0, 'false_positive': 1, 'false_negative': 1},
   {'correct': 0, 'false_positive': 0, 'false_negative': 1},
   {'correct': 1, 'false_positive': 0, 'false_negative': 4},
   {'correct': 0, 'false_positive': 0, 'false_negative': 0},
   {'correct': 0, 'false_positive': 0, 

now build the expansion patterns from the training data

In [9]:
seeds.build_patterns(train)

10297


and execute them against the data
Fair warning, this is not optimized :)
The patterns class maintains a list of its results for each iteration of run in its results object, so it will output the change between each evaluation run.
Each call to run, appends the best 10 new seeds and also calls evaluate.

In [10]:
result = seeds.run(train)

100%|██████████| 10297/10297 [13:18<00:00, 12.90it/s]


Entity Set Precision: 0.7121212121212122
Entity Set Recall: 0.3375224416517056
Entity Set F1: 0.4579780755176614
Entity Token Set Precision: 0.9411764705882353
Entity Token Set Recall: 0.376101860920666
Entity Token Set F1: 0.5374387683694891
      Partial Recall changed from 0.36 to 0.38, a change of 0.01
           Precision changed from 0.72 to 0.71, a change of -0.00
              Recall changed from 0.32 to 0.34, a change of 0.02
                  F1 changed from 0.44 to 0.46, a change of 0.02
   Partial Precision changed from 0.95 to 0.94, a change of -0.01
          Partial F1 changed from 0.52 to 0.54, a change of 0.01
{'F1': 0.4579780755176614, 'Precision': 0.7121212121212122, 'Recall': 0.3375224416517056, 'Partial F1': 0.5374387683694891, 'Partial Recall': 0.376101860920666, 'Partial Precision': 0.9411764705882353, 'New Candidates': ['rejected ballots', 'voter', 'constituency association', 'relevant electoral division', 'statement', 'election period', 'election process', 'pol

While the first run is fairly slow, it will memoize the seeds and subsequent runs will be much faster. 
The routine will run for a long time if not provided with a stop point, (it takes a long time to converge). 

In [ ]:
 for _ in range(50):
    seeds.build_patterns(train)
    seeds.run(train)

11247


100%|██████████| 11247/11247 [01:30<00:00, 124.31it/s]


Entity Set Precision: 0.7027972027972028
Entity Set Recall: 0.3608617594254937
Entity Set F1: 0.4768683274021353
Entity Token Set Precision: 0.9216589861751152
Entity Token Set Recall: 0.3917727717923604
Entity Token Set F1: 0.549828178694158
      Partial Recall changed from 0.38 to 0.39, a change of 0.02
           Precision changed from 0.71 to 0.70, a change of -0.01
              Recall changed from 0.34 to 0.36, a change of 0.02
                  F1 changed from 0.46 to 0.48, a change of 0.02
   Partial Precision changed from 0.94 to 0.92, a change of -0.02
          Partial F1 changed from 0.54 to 0.55, a change of 0.01
{'F1': 0.4768683274021353, 'Precision': 0.7027972027972028, 'Recall': 0.3608617594254937, 'Partial F1': 0.549828178694158, 'Partial Recall': 0.3917727717923604, 'Partial Precision': 0.9216589861751152, 'New Candidates': ['voters ballot', 'ballot box', 'ballots', 'election clerk', 'official', 'candidates official', 'registration officer', 'notice', 'report', 'surv

100%|██████████| 12414/12414 [01:49<00:00, 113.18it/s]


Entity Set Precision: 0.6845637583892618
Entity Set Recall: 0.36624775583482944
Entity Set F1: 0.4771929824561404
Entity Token Set Precision: 0.9042316258351893
Entity Token Set Recall: 0.39764936336924583
Entity Token Set F1: 0.5523809523809523
      Partial Recall changed from 0.39 to 0.40, a change of 0.01
           Precision changed from 0.70 to 0.68, a change of -0.02
              Recall changed from 0.36 to 0.37, a change of 0.01
                  F1 changed from 0.48 to 0.48, a change of 0.00
   Partial Precision changed from 0.92 to 0.90, a change of -0.02
          Partial F1 changed from 0.55 to 0.55, a change of 0.00
{'F1': 0.4771929824561404, 'Precision': 0.6845637583892618, 'Recall': 0.36624775583482944, 'Partial F1': 0.5523809523809523, 'Partial Recall': 0.39764936336924583, 'Partial Precision': 0.9042316258351893, 'New Candidates': ['court', 'ballot boxes', 'official agents', 'voter place', 'polling subdivisions', 'vote', 'paper ballot', 'particular polling', 'election

100%|██████████| 12939/12939 [01:01<00:00, 210.52it/s]


Entity Set Precision: 0.6743421052631579
Entity Set Recall: 0.36804308797127466
Entity Set F1: 0.47619047619047616
Entity Token Set Precision: 0.8991228070175439
Entity Token Set Recall: 0.4015670910871694
Entity Token Set F1: 0.5551794177386594
      Partial Recall changed from 0.40 to 0.40, a change of 0.00
           Precision changed from 0.68 to 0.67, a change of -0.01
              Recall changed from 0.37 to 0.37, a change of 0.00
                  F1 changed from 0.48 to 0.48, a change of -0.00
   Partial Precision changed from 0.90 to 0.90, a change of -0.01
          Partial F1 changed from 0.55 to 0.56, a change of 0.00
{'F1': 0.47619047619047616, 'Precision': 0.6743421052631579, 'Recall': 0.36804308797127466, 'Partial F1': 0.5551794177386594, 'Partial Recall': 0.4015670910871694, 'Partial Precision': 0.8991228070175439, 'New Candidates': ['election survey', 'campaign worker', 'candidates names', 'annual report', 'voters intention', 'petitioner', 'nomination paper', 'persons

100%|██████████| 13344/13344 [00:52<00:00, 252.91it/s]


Entity Set Precision: 0.5810055865921788
Entity Set Recall: 0.3734290843806104
Entity Set F1: 0.4546448087431694
Entity Token Set Precision: 0.8326848249027238
Entity Token Set Recall: 0.41919686581782567
Entity Token Set F1: 0.5576547231270358
      Partial Recall changed from 0.40 to 0.42, a change of 0.02
           Precision changed from 0.67 to 0.58, a change of -0.09
              Recall changed from 0.37 to 0.37, a change of 0.01
                  F1 changed from 0.48 to 0.45, a change of -0.02
   Partial Precision changed from 0.90 to 0.83, a change of -0.07
          Partial F1 changed from 0.56 to 0.56, a change of 0.00
{'F1': 0.4546448087431694, 'Precision': 0.5810055865921788, 'Recall': 0.3734290843806104, 'Partial F1': 0.5576547231270358, 'Partial Recall': 0.41919686581782567, 'Partial Precision': 0.8326848249027238, 'New Candidates': ['nomination papers', 'electors name', 'information', 'supply vote', 'persons name', 'deputy', 'information officer', 'person', 'personal in

  3%|▎         | 399/15100 [00:04<03:06, 78.62it/s] 

# Self-References

We first detect all the self refferences in the dataframe using the graph G

In [ ]:
find_referrals_in_df(train, G)

Then we validate the initial references, (this will also add them to the graph G)

In [ ]:
set_candidates(G)
train['self_references_validated']= train.apply(lambda row: {x for x in validate(row['node'], row['self_references'], G)}, axis=1)

The initial evaluation data was saved without the node text, so this helper method resolves the node from the text of the data
it was also saved with a different punctuation removal that stripped the parentheses from the data, so they need to restored. 
The below code is less than beautiful, but it performs those patching functions.

In [ ]:
def nodefromText(text):
    text = remove_punctuation(text[getBreak(text):].strip().lower()).replace('(', '').replace(')', '')
    for node in G.nodes:
        try:
            
            if remove_punctuation(G.nodes[node]['text']).lower().strip().replace('(', '').replace(')', '') == text:
                return node
        except KeyError:
            continue
eval_data['node'] = eval_data.raw_text.apply(nodefromText)

In [ ]:
eval_data['self_references'] = eval_data.raw_text.apply(lambda x: find_possible_referrals(x.lower()))
eval_data['self_references_validated']= eval_data.apply(lambda row: {x for x in validate(row['node'], row['self_references'], G)}, axis=1)
eval_data['self_references_validated'] = eval_data['self_references_validated'].apply(lambda x: {remove_punctuation(y) for y in x})

In [ ]:
r = evaluate_entity_sets(eval_data['self_references_validated'])

def proc(token_list):
    r = set()
    for ent in token_list:
        r.update(ent.split())
    return r
eval_data['self_references_validated_tokens']=eval_data['self_references_validated'].apply(proc)
rtok = evaluate_entity_token_sets(eval_data['self_references_validated_tokens'])
